# Agent-Base Modeling - SIRV

We are living unusual times, where humankind is under the attack of a microscopic creature that threatens our health (physical and mental), economy, social goods, etc. The number of variables involved and the complexity of their relationships is making the people freak out to understand the implications of his local actions in future global scenarios, from short to midterms. But how could we project those future scenarios? How does the government know what will be the positive effects of taking such protective measures?  

Well, there are many possible answers to this question. First, we have to define that a society living over territory is a Complex and Dynamical System. Such kinds of systems have many interacting variables, presents nonlinear behavior and its properties evolve with the time. Its behavior is generally stochastic, it may depend also on its initial conditions, it can be affected by neighbor societies (with different policies and dynamics) and it can show emergence.

**Agent-Based Modeling** are a good choice to simulate these dynamic complex systems due to its simplicity of implementation and accurate results when compared with real data of past events. And also because we can design scenarios by performing interventions on the simulation environmental variables and measure its effects.

Now we going to assume Discrete Events for our simulation, meaning that the time and the components of the system are discrete and atomic. More specifically, the Agent-Based Modeling (ABM) is composed of a population of agents, within a loop where the agents run and interact. Each agent is considered autonomous, in the sense that it has its internal state, but the population of agents shares a common environment. The state variables drive the behavior of each agent and they must represent the characteristics of the problem. 

ABM is employed in the simulation of complex dynamical systems using the individual states of its components to detect global behaviors that emerge due to the interactions between the agents over the iterations. Then, its main goal is to simulate the temporal evolution of the system, storing statistics derived from the internal states of the agents in each iteration. ABS allows us to simulate systems with intricate nonlinear variable relationships, complex conditions and restrictions that may be hard to describe mathematically.

**If** the main strength of ABM is its flexibility, on the other hand, its main drawback is the high computational cost. It is much cheaper to have a closed-form equation (ODE’s in the case of dynamical systems), or even perform stochastic simulations with Monte Carlo Markov Chain models. However, as said before, not always we could achieve such.

Attentively: Petr, and nio Silva. “Agent Based Simulation of COVID-19 Health and Economical Effects.” Medium, Towards Data Science, 14 Apr. 2020, [towardsdatascience.com/agent-based-simulation-of-covid-19-health-and-economical-effects-6aa4ae0ff397](towardsdatascience.com/agent-based-simulation-of-covid-19-health-and-economical-effects-6aa4ae0ff397).

## Epidemic Simulation Overview

First of all, let’s decide what we want to simulate. The contagious spreading caused by the circulation and the effects of social isolation are our primary goals.

### Contagious Spreading

Following the **SIRV** model, each agent must be in one of these situations: **Susceptible**, **Infected**, **Recovered** and **Vaccinated**. Each simulation has an initial percentage of infected people, and the remaining population is composed of susceptible individuals. The status also contains the **Death** status, for those agents who develop severe symptoms of the SARS-COVID-2 and did not resist.

The **main assumption** of the contagious spreading is the interaction of the agents by proximity or contact. So, as higher is the mobility of a person, greater is the probability that he approaches an infected person and gets also infected. Each simulation will have a **Contagion Distance**, the minimal distance that two agents have to be to occur the viral transmit. In addition, for viral transmission to occur each individual has a probability of transmission $\beta$ uniformly distributed variate on $[\beta_{min}, \beta_{max}]$, this reflects the level of hygiene of an individual. $\beta$ was chosen as an agent parameter instead of a model parameter since in a realistic scenario, the actual virus transmission would depend on the $\beta$ value of both agents. However, only the $\beta$ value of the infected individual was considered here for simplicity.

After a period of infection $\tau$, an individual may recover or die. However, upon recovery, the individual is still not safe. A probability of reinfection is defined. Although this is still under debate, the model allows evaluating the possibility of future scenarios given these rules.

### Mobility Patterns

The simulation is performed in a continuous bi-dimensional **environment** shared by all the agents. Each agent is randomly initialized inside this environment such as $(x, y) \sim U(0,1)$ and with a velocity assigned through a function $f(s) = sincos(2\pi \cdot v) \cdot s$, such that $v \sim U(0,1)$ and $s$ is a speed constant   between all agents.
 
The proximity between two agents $a_1$ and $a_2$ is computed using the CityBlock distance, such as $dist(a_1, a_2)=|x_{a_1} - x_{a_2}| + |y_{a_1} - y_{a_2}|$.

To simulate the contacts, an **elastic collision** function is resolved between the agents $a_1$ and $a_2$. They are assumed to be disks of equal size touching tangentially. Their velocities are adjusted for an elastic collision happening between them. Collision only happens if both disks face each other, to avoid collision-after-collision.

One of the two agents can have infinite **"mass"**, and then acts as an immovable object (isolated individuals) that specularly reflects the other agent. In this case of course momentum is not conserved, but kinetic energy is still conserved.

Automatically all dead agents are removed from the environment.

### Interventions

#### Social isolation

To simulate the impact of social distancing, different types of emergency closings were proposed: i) for an indefinite period, ii) with a defined period of time and, finally, iii) with periodic repetitions. For all cases, given a percentage of isolation from the population, their corresponding individuals are chosen at random. In the case of periodic repetitions of isolation, new individuals are chosen in each period.

#### Vaccination

Society hopes that, at some point, a vaccine will be developed in order to contend with the virus; providing immunity to its population. However, this may take time. To make an approximation of how the pandemic would evolve given the invention of a vaccine, the possibility of being a vaccinated agent was added. It is known that, in principle, not all vaccines are equally effective in the population. However, it was considered a complete effectiveness for simplicity, that is, once vaccinated, the individual cannot be infected again.

### Economics impacts

A secondary goal is to simulate the impact caused in the economy — especially in each agent’s **income** and **wealth** — by the mobility restrictions. The basic **assumption** of the economical dynamics of this simulation is: the **income** of the agents is a function of its mobility. In the real world, there are many cases where this condition is not true, and people still working at home. But unfortunately, this is not always the case.

## Load dependencies

In [1]:
using DrWatson
@quickactivate(@__DIR__)

using 
    Agents,
    AgentsPlots,
    Base.Iterators,
    CSV,
    Dates,
    DataFrames,
    DataFramesMeta,
    Distributed,
    Gadfly,
    Plots,
    Random,
    StatsBase

In [2]:
src_folder = srcdir("social_distancing")
src_files = ["Agent.jl", "World.jl", "Utils.jl"]
for src_file=src_files include(joinpath(src_folder, src_file)) end

## Simulations directories

In [3]:
out_dir = mkpath(datadir("sims", "social_distancing"))
individual_sir_model = mkpath(joinpath(out_dir, "individual"))
β_effect_dir = mkpath(joinpath(out_dir, "β_effect"))
grid_experiments_dir = mkpath(joinpath(out_dir, "grid"));

## Running AMB simulations

<figure>
    <img src="../plots/social_distancing/periodic_lockdown_with_vaccination_slow_β=50-50.gif" alt="Drawing" style="width: 600px;"/>
</figure>

### Overview SIRV model

Demonstratively, **figure 1.0.** shows a simulation of the model of individual agents with periodic social isolation with 50% of the population from day 24, with durations of 24 days and intervals between isolations of 14 days. In addition, the invention included a vaccine from day 240.

It is interesting to note how, after a time of beginning periods of social isolation, infected and recovered individuals begin to oscillate; a product of the built-in probability of reinfection and periodic isolations. However, when the vaccine is added to the system, little by little the disease begins to disappear. This behavior could be important, if it is shown that individuals can be reinfected.

<figure>
    <img src="../plots/social_distancing/fraction_isolated=0.5_interval_between_locks=336_nsteps=12000_periodic_lockdown=true_start_lockdown=720_start_vaccination=7200_stop_lockdown_after=720_βmax=4_βmin=0.png" alt="Drawing" style="width: 600px;"/>
    <figcaption><b>Figure 1.0.</b> Dynamics of the individual SIRV agent model.</figcaption>
</figure>

### Effect of $\beta$ amplitude on survival

Unlike models where it is assumed that there is a constant infection rate for all individuals, this model aims to assess the effect of hygiene measures on each individual. If an individual has very poor hygiene measures, they are more likely to infect another person. In a model closer to reality, the probability of infection should depend on the hygiene measures of both individuals, however, for simplicity, only those from the affected individual were considered.

The hygiene measures of a population can determine a high spread of the disease. The transmission probability $\beta$, is symbolic of how hygienic a person is; a value of one equals nothing and a value of zero is flawless. **Figure 2.0** shows a combination of different amplitudes of hygiene measures in a population; a range between impeccability and doom.

It was found that, in general, having a narrow range distribution with values between $\beta_{max}$ and $\beta_{min}$ of 0.5 is very similar to having a wide distribution bounded by low values of $\beta_{ min}$. However, when both $\beta$$s$ pass values 0.5 (distributions of $\beta$$s$ narrow) the population decays with greater weight. Therefore, it could be said that, in fact, having good hygiene measures in your population could help to not continue spreading the virus and that, assuming that the hygiene of the population is distributed uniformly, it may not be so bad, but yes upgradeable.


#### Parameters

In [4]:
β_args = Dict(
    :nsteps => 500steps_per_day,
    :start_lockdown => 30steps_per_day,
    :stop_lockdown_after => 30steps_per_day,
    :periodic_lockdown => true,
    :interval_between_locks => 14steps_per_day,
    :fraction_isolated => 0.5,
    :start_vaccination => 300steps_per_day,
    :βmax => collect(0:0.1:1.0),
    :βmin => collect(0:0.1:1.0)
)

dict_β_args = dict_list(β_args)
dict_β_args = filter(x->x[:βmax] ≥ x[:βmin], dict_β_args); length(dict_β_args)

66

#### Run

In [5]:
results = pmap(x->simulate(x; out_dir = β_effect_dir, return_results = true)[end, Not(:step)], dict_β_args);

#### Preprocessing

In [6]:
df = DataFrame(dict_β_args)[!, [:βmax, :βmin]]
df[!, :N] = map(sum, results)
CSV.write(datadir("exp_pro", "effect_β_amplitude_on_survival.csv"), df);

#### Results

<figure>
    <img src="../plots/social_distancing/effect_β_amplitude_on_survival.png" alt="Drawing" style="width: 600px;"/>
    <figcaption><b>Figure 2.0.</b>Effect of the distribution of hygiene measures on the population.<figcaption>
</figure>

### Model performance evaluation

The main objectives of the model are to evaluate the contagious spread caused by circulation and the effects of social isolation. In the previous section, we discussed the effects on the extent of the spread of infection associated with the hygiene measures of each individual. Therefore, the effectiveness of different interventions was evaluated with the objective of reducing the number of deaths.

In order to obtain the different simulations, the Cartesian product of different parameters such as start and end times of isolates, whether they were periodic or not, different proportions of individuals in isolation, the possibility of inventing a vaccine and, finally, different ranges of $\beta$$s$.

In **Figure 3.0.** The behavior of individuals over time is shown in each of the simulations performed, highlighting the top 3 in the number of deaths was minimum (best) and maximum (worst), and those outside the top (others). In turn, the tops are shown for each evaluated $\beta_{min} $ (0.0 and 0.5) with a $\beta_{max}$ of 0.8. In general, the best models were characterized by isolates from day thirty and with a duration of one and two months, without repetition and with the invention of a vaccine (**Table 1.0.**). However, it is noteworthy that one of the best models did not need the invention of a vaccine to be the best (id 186), but an isolation of two months in conjunction with a wider range of betas, which as seen in the previous section, helps reduce the number of deaths. For their part, the worst models were noted for doing nothing, not isolating, not vaccinating, and with a narrow betas range skewed to high values.

#### Parameters

In [7]:
n_steps = 500 * steps_per_day

general_args = Dict(
    :N => 1000,
    :nsteps => n_steps,
    :start_lockdown => [n_steps + 1, 30 * steps_per_day, 100 * steps_per_day],
    :stop_lockdown_after => [30 * steps_per_day, 60 * steps_per_day],
    :periodic_lockdown => [false, true],
    :interval_between_locks => [21 * steps_per_day, 60 * steps_per_day],
    :fraction_isolated => [0.0, 0.5],
    :start_vaccination => [n_steps + 1, 300 * steps_per_day],
    :vaccination_probability => 0.001,
    :vaccine_effectiveness => 1.0,
    :βmin => [0.0, 0.5],
    :death_rate => 0.044,
)

dicts_args = dict_list(general_args)
dict_list_count(general_args)

192

#### Run

In [8]:
pmap(x->simulate(x; out_dir = grid_experiments_dir, return_results = false), dicts_args);

#### Results

<figure>
    <img src="../plots/social_distancing/ABM_proportion_curves_betas.jpg" alt="Drawing" style="width: 600px;"/>
    <figcaption><b>Figure 3.0.</b> Model performance evaluation with objective function as number of living individuals.<figcaption>
</figure>

<figure>
    <img src="../plots/social_distancing/better-worst_performance_parameters.png" alt="Drawing" style="width: 950px;"/>
    <figcaption><b>Table 1.0.</b> Better and worst models.<figcaption>
</figure>
        

## Discussion

Given these results, it is interesting to ask what the future will be like without the invention of a vaccine and how, by following sanitary measures together with obeying social distancing, it could help to reduce the number of deaths significantly. In turn, it will be interesting to determine which model is better based on deaths along with the impact on the economy. Not everyone can do home office, only a small fraction of the population can afford to do it. At the moment, obeying sanitary measures and social isolation are the best option, washing your hands, wearing face masks, keeping your distance and staying at home should be a momentary lifestyle for those who can afford it and for those who do not, it will be interesting to see the socioeconomic plans that are developed to deal with the health and economic problems. Thus concluding that carrying out interventions that minimize the number of deaths while maximizing the economy, they would be adequate to proceed in a country, this is because there are multiple external causes that depend on the economy, including access to food or health. Without access to these primary services, part of the population could suffer collateral damage, including death. 